
# Fuzzy Inference Systems - XAI 2025

Author: Arthur Babey

Based on the work of Thibault Schowing, Hector Satizabal and Gary Marigliano

- Professor: Carlos Peña (<a href="mailto:carlos.pena@heig-vd.ch">carlos.pena@heig-vd.ch</a>)
- Assistant 2025: 
    - Arthur Babey (<a href="mailto:arthur.babey@heig-vd.ch">arthur.babey@heig-vd.ch</a>)
    
Due: 26 Mars 2025, 23h59

* We will mainly use an implementation of a library : lfa_toolbox in order to build, modify and visualize fuzzy system.
* You can have a look here at [lfa_toolbox](https://github.com/arthurbabey/lfa_toolbox) and dive into the code if needed.

The code is structured like this:
```
lfa_toolbox
├── core
└── view
```

* `core` contains core classes like membership_functions, fuzzy_rules,...
* `view` contains classes used to display what the core classes do.
* GO LOOK in the code if you have any doubts to have a more precise idea of how it works. The code is rich but relatively simple!

## LFA Tools 

We will also use **LFA Tools**. This library makes it easy to visualize fuzzy systems, predictions, and all the characteristics of our fuzzy logic models.  

To install it, run:  

``` python
pip install git+https://github.com/arthurbabey/lfa_toolbox.git
```

---

# Build a fuzzy system step by step

## Part one:  simple example of a complete Fuzzy Inference System

In this lab you will first analyse an existing fuzzy system solving a relatively simple problem. Take your time to analyse the example and support thoroughly as you will be expected to understand it for part two. 

- Start first by reading and understanding the "**Fuzzy_Logic_Tutorial.ipynb**" notebook joined with this lab. It contains a detailed example with all the steps to create a fuzzy system from A to Z. 

- Read [this tutorial](https://ch.mathworks.com/help/fuzzy/fuzzy-inference-process.html) mentioned in the above mentioned notebook. It gives a greatly summarized explanation with comprehensive graphics, even though the vocabulary might be a little bit different.

- Make sure to be able to answer the questions bellow. You can write the answer / copy paste the definition here if it helps you: 

    - What is a linguist variable ?
    - ... a membership function ?
    - ... a Antecedent ?
    - ... a rule ?
    - ... the aggregation function ?
    - ... the defuzzification ?
    - ... the difference between a Mamdani and a Singleton Fuzzy System ?
    - ... a default rule ?

---
## Part two: implementation of the car acceleration problem

In [ ]:
%matplotlib ipympl

import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt

from lfa_toolbox.core.lv.linguistic_variable import LinguisticVariable
from lfa_toolbox.core.mf.lin_piece_wise_mf import LinPWMF
from lfa_toolbox.core.mf.trap_mf import TrapMF
from lfa_toolbox.core.mf.triangular_mf import TriangularMF
from lfa_toolbox.core.mf.singleton_mf import SingletonMF

from lfa_toolbox.core.rules.fuzzy_rule import FuzzyRule, Consequent, Antecedent
from lfa_toolbox.core.rules.default_fuzzy_rule import DefaultFuzzyRule
from lfa_toolbox.core.fis.fis import FIS, OR_max, AND_min, COA_func, MIN
from lfa_toolbox.core.fis.singleton_fis import SingletonFIS

from lfa_toolbox.view.lv_viewer import LinguisticVariableViewer
from lfa_toolbox.view.fis_viewer import FISViewer
from lfa_toolbox.view.fis_surface import show_surface




### A) Basic implementation (Mamdani model)

In this exercise we will look into the car acceleration problem that you have seen in the lectures and transform it, step by step, into an **interpretable fuzzy system**. 

**Tip: While you are doing the exercises, you can compare your results with the surface plots shown in the lecture notes.**

The car acceleration problem is a problem where you need to decide which **action** between "**push**", "**release**" or "**nothing**" you must perform knowing the **speed** and the **speed change** (acceleration) of your car to maintain the same speed.

We can model this problem with this initial proposition:


<img src="data:image/png;base64, ...
" />

The figure above tells you which combination of inputs leads to which output. The goal here is to first implement a basic FIS and then to reduce its complexity to make it more interpretable.

**Read the code bellow that implements a basic Fuzzy Inference System and complete the comments**.

In [ ]:
# LV car_speed
car_speed = LinguisticVariable(name='speed', ling_values_dict={
    'slow': LinPWMF([-0.2, 1], [0, 0]),
    'ok': LinPWMF([-0.2, 0], [0, 1], [0.15, 0]),
    'fast': LinPWMF([0, 0], [0.15, 1], [1, 1])
})

# LV speed change
car_acc = LinguisticVariable(name='speed_change', ling_values_dict={
    'slowing': LinPWMF([-0.3, 1], [0, 0]),
    'constant': LinPWMF([-0.3, 0], [0, 1], [0.3, 0]),
    'rising': LinPWMF([0, 0], [0.3, 1])
})

# LV action
car_action = LinguisticVariable(name='action', ling_values_dict={
    'release': LinPWMF([-1, 1], [0, 0]),
    'nothing': LinPWMF([-1, 0], [0, 1], [1, 0]),
    'push': LinPWMF([0, 0], [1, 1])
})

# Rules
car_rules = [
    FuzzyRule(ant_act_func=AND_min,
              ants=[Antecedent(car_speed, 'fast'),
                    Antecedent(car_acc, 'constant')],
              cons=[Consequent(car_action, 'release')], impl_func=MIN),

    FuzzyRule(ant_act_func=AND_min,
              ants=[Antecedent(car_speed, 'fast'),
                    Antecedent(car_acc, 'rising')],
              cons=[Consequent(car_action, 'release')], impl_func=MIN),

    FuzzyRule(ant_act_func=AND_min,
              ants=[Antecedent(car_speed, 'slow'),
                    Antecedent(car_acc, 'constant')],
              cons=[Consequent(car_action, 'push')], impl_func=MIN),

    FuzzyRule(ant_act_func=AND_min,
              ants=[Antecedent(car_speed, 'slow'),
                    Antecedent(car_acc, 'slowing')],
              cons=[Consequent(car_action, 'push')], impl_func=MIN),

    FuzzyRule(ant_act_func=AND_min,
              ants=[Antecedent(car_speed, 'fast'),
                    Antecedent(car_acc, 'slowing')],
              cons=[Consequent(car_action, 'nothing')], impl_func=MIN),

    FuzzyRule(ant_act_func=AND_min,
              ants=[Antecedent(car_speed, 'ok'),
                    Antecedent(car_acc, 'slowing')],
              cons=[Consequent(car_action, 'nothing')], impl_func=MIN),

    FuzzyRule(ant_act_func=AND_min,
              ants=[Antecedent(car_speed, 'ok'),
                    Antecedent(car_acc, 'constant')],
              cons=[Consequent(car_action, 'nothing')], impl_func=MIN),

    FuzzyRule(ant_act_func=AND_min,
              ants=[Antecedent(car_speed, 'ok'),
                    Antecedent(car_acc, 'rising')],
              cons=[Consequent(car_action, 'nothing')], impl_func=MIN),

    FuzzyRule(ant_act_func=AND_min,
              ants=[Antecedent(car_speed, 'slow'),
                    Antecedent(car_acc, 'rising')],
              cons=[Consequent(car_action, 'nothing')], impl_func=MIN)
]

# Fuzzy Systems
fis = FIS(rules=car_rules, aggr_func=np.max, defuzz_func=COA_func)

# Surface 
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='3d'))
show_surface(fis, ax=ax, title="Car Problem Mamdani", x_label="speed", y_label="speed_change", z_label="action",
                 n_pts=15, x_range=(-1, 1), y_range=(-1, 1), z_range=(-1, 1))

## Questions 

- (1 pt) Why this system is a Mamdani FIS ?

- (1 pt) What is showing the surface plot ? is it before or after defuzzification ?

*Réponse:*

---
*Réservé pour corrections*
<div class="alert alert-block alert-info">
<b>Corrections: /2 </b>
</div>

Commentaires: 

### B) Simplification with a Singleton model


The above example is a **Mamdani FIS**. In this section, you are asked to make a **Singleton FIS**. You can find the definition and example in the tutorial notebook joined with this lab. 



* Copy and paste the above implemented FIS system.
* Replace the defined linguistic variable __car_action__ with the following code:

``` python
car_action = LinguisticVariable(name='action', ling_values_dict={
    'release': SingletonMF(-1),
    'nothing': SingletonMF(0),
    'push': SingletonMF(1)
})
```
* Convert Mamdani FIS to Singleton. To achieve this goal, replace __fis__ variable with the follwoing code:
``` python
fis = SingletonFIS(rules=car_rules)
```

* Use `show_surface()` function to display the result and compare it with the Mamdani FIS in a few words.
* Change the comments and the title of the surface plot accordingly.



In [ ]:
# Your code here




---
*Réservé pour corrections*
<div class="alert alert-block alert-info">
<b>Corrections: /6 </b>
</div>

Commentaires: 

### C) Simplification with a Singleton Model, Default Rule and Not Conditions


Use the FIS from the previous section (singleton & default rule), and change fuzzy rules to add **_NOT conditions_** in order to respect the following table. There are 3 distinct area (P, N and R), so there will be 3 rules (2 fuzzy rules and 1 default fuzzy rule).

<img src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAUoAAAB5CAYAAABfsGiyAAAABHNCSVQICAgIfAhkiAAACCZJREFUeJzt3X1sXXUdgPHn3NvbN8roytwGe2FzIxvvGHDAhgq+gAMTYIpCRoIgIIphoIZIItqILxAFFuMUxJFJAoEphIi8CChB2SS8jJcJxbG5zQ7moIO+rKW97b3HP1a021p+2+h67j17Pn/dc3OafLecPD33d885jeI4jpEkDakCIIqipOeQpJJV8d6LxgSHkLTrGhuTnmDvkUl6AEkqdYZSkgIMpSQFGEpJCjCUkhRgKCUpwFBKUoChlKQAQylJARXhXcpTVA/TTobcUL8KitD9H2hdC22boOgd75KGkN5QVkLDLDjsQpg8tv/NPnhnBWzeBMUa2G8KNEyFeDU8ez0svwM680lOLakUpTaUxTfh6avhpT/BZY9BXRY2fQtu/TkUBuxXOxvm3g6zF8MRc+E350G7sZQ0QOrXKHuboasHKELbc9tGEqBrOdx/DmzugX3nwemngM9SkjRQ6kMZ56FQ7H89xDpkfiWsXAtkYcLFUDli00kqB6kP5U4pQHvX1pfZUf6nSNqWTQCogUn9X/h0LoPeZKeRVGIMJTD6fDh0AsTt8Mxi6Et6IEklJbXfeodE1TBqGkz+Mpy6AKry8PKV8OzapCeTVGr2nlBmYMYy+P572zEUOqBjAzT/El5YDKtWgtedS9re3hPKIqz6JCx9qn87hoLXS0raCXtPKIG4Fwo9SU8hqdz4ZY4kBaQ/lANus/HPl0vaHakPZVQDuSyQger6pKeRVI7SHcoIGk6DUdVbNxsugLq9alVW0nBIbTYqZsKX7oOpB0O2/yN33Ty4fD288TAsvQS6tn9ChiQNIrWh7HsV7piZ9BSS0iDdH70laRgYSkkKMJSSFGAoJSnAUEpSgKGUpABDKUkBhlKSAgylJAUYSkkKMJSSFBDFcRxHUQSNSY8iSaXJM0pJCjCUkhRgKCUpwFBKUoChlKQAQylJAYZSkgIMpSQFGEpJCjCUkhSQ2j9Xq/SpqoNTJ0Fu4JtFeHoNNPdtu+/oA+Gk/XY8E3hzPfytaw8PqtQxlCobmRwcchDMPxqOqOp/M4bn/gwnLoPuAfvmauCwg+Dso+HIKujrht8/D/duSGJylTsfiqGyM/Ej8MrnoDUPE6sh7oav3Qy/bttx3wlHwStnwIrfwaeaoDjy4yoFXKNU2WndBO19cOeD8FofZKrh2rkwPhpk3zehtQ9e2GwktfsMpcpOsQD5GDo2wqUvQR4YezBcN3XHA7pYgN4itFtJfQCGUuUrhr/+Be7bAmTgnNNhVlXwp6RdZihV1gqd8M1HoSWGqga45XioTXoopY6hVNl7/WVobN66Bnn4CXDR6KQnUtoYSpW/Atz2R3gxD5kquOZ0mOCRrWHk4aRUePctuPgZeBcYMxVumA7ZpIdSahhKpcbzT8LtrRBn4Ky5MKc66YmUFoZSqVHshmsego1FqKyHm+dA3SDXVkq7ylAqVd5aDd9eAwVgxiz4+v5JT6Q0MJQqPxFEDHHwFuGeB2FZN2Qq4arPwjiPcn1AHkIqO9kKqM7C2CEe6ZJvhUuehI4Yavf147c+OEOp8hLBsUfC6Ar49KFQN8Ruq56GX2yGeESHU1oZSpWN+snw9wXwyEehCpj5cWheAPcet3V7oLgXrr8f1hWSmFRp42PWJCnAM0pJCjCUkhRgKCUpwFBKUoChlKQAQylJAYZSkgIMpSQFGEpJCjCUkhQwxPNXlDqNC5OeQCpbnlFKUoChlKQAQylJAYZSkgIMpSQFGEpJCjCUkhRgKCUpwFBKUoChlKQAQylJAd7rLWnYZOubOeXkt6l931OwiLivkpYNo1m3roENm3MUR2rA3WQoJQ2bTGUn02f9g7MvXMGJYwtEAHGGzjfG8eKr9WxuryTeZwsHTHibQ2a0UpfJseax4/nZjz/D4if2pTfpf8AQojiO4yiKoDHpUbRH+fQgjZiY+pOW0PTYi4zP5mhqvJxZP5jElnjbffaZtoqv/vROfnJmG7meMSw+60oufXgfCkmN/T5co5Q0zCK6mqfwVg9QrOK1Rz+0XSS37tO5ZgY3nf8VblqdI6puYf6NTzK9KoFxd4KhlDTsivka8jux8Bh3jGfpE3X0AtUH/pNpdTsUtSQYSkkJKlI1qkAGiPPVdPRFSQ80KEMpKTHZcau44mOdZMnQsnwOTR1JTzQ4QykpATG101/hu0vv4vMHFOleewwLLptJS4leJ+TlQZL2nOhdTli4mEfb/v+ROsr1MOaAVj48tZP8uon84YbZ/PDaY1nRVrrnbYZS0p4T5/jX0jksbKoEIBr1b65e9AhHjY7pWTebM477Ao+/U5rrkgMZSkl7UAWbls/kgeXV/dszWFm7jmdveZUxBz3FkhsP45iLDqWlFC+eHKB0z3UlpVCW9b89lyseqKMvKjB5/l0sOqu95M/YDKWkkdW3H3df8kXu2ZiFXDvzFt3NuRNL+5TSUEoaflGR91YeM4MsQfZtOpxvXHQMzfmIirFNLFzyFFMrR3TCXWIoJQ27bE0HNVkg00d9fYEdW5mh5eEzueC2BnriIg0n388dCzZSW6Lf6xhKScMr6mXaaSuZVA2QZ/oFaxg32CJksZbHv3Mev3otR5zp5vgfLeLeq1Yzsar0bmM0lJKGTc3MR3ioqZEVP3udugigyPh5S1i9/jqeuHUV+29XnGLbFL43/xO83BUR5bZw6nWLWLthEZdNKa01Sx+ztrfwMWvSbvOMUpICDKUkBRhKSQowlJIUYCglKcBQSlKAoZSkAEMpSQGGUpICDKUkBfzvFsY4Lr0b0SWpFPwXAxXvkmrRh/cAAAAASUVORK5CYII=
" />

* Copy and edit your singleton & default rule FIS to match the above figure.
* Use `show_surface()` function to display the result and compare it with the Mamdani model in a few words.
* Change title of the surface plot accordingly.

**Tip.** The following example shows how to make an antecedent negative:

- Car speed is fast: 
``` python
Antecedent(car_speed, 'fast')
```

- Car speed is **not** fast: 
``` python
Antecedent(car_speed, 'fast', is_not=True)
```

In [ ]:
# Your code here




### Compare the Different Models 

- (7 pts) Analyze and compare the 3 variants of the car problem with a few words. Specifically, ask yourself about the interpretability aspect, the performance/accuracy. What are the pros and the cons of each FIS compared to the original one (first one/Mamdani).
  
- (2pt) Which model (model A, B, or C) do you think is the simplest to be built by an algorithm? Justify your answer. 


*Réponse:*


---
*Réservé pour corrections*
<div class="alert alert-block alert-info">
<b>Corrections: /15 </b>
</div>

Commentaires: 

## Visualizing a fuzzy inference 

Let's take the "Singleton Model, Default Rule and Not Conditions" fuzzy system from the last question and visualize an inference. 
(If you did not find it, use the Mamdani model instead)


In [ ]:
fis = SingletonFIS(rules=car_rules, default_rule = def_rule)

# data point
input_values = {'speed': 1, 'speed_change': 0.22}
# prediction
fis.predict(input_values)
# visualisation
fisv = FISViewer(fis)
fisv.show()

## Questions : 

- (1 pt) What is the system’s final output or prediction?

- (2 pt) Which rules are triggered and why? (which variables and membership functions lead to each rule’s activation?)

- (2 pt) Can you find a specific data point (speed, speed_change) such that the system’s output is “push” (i.e., action = 1)? Show it. 

In [ ]:
# Your code here


*Réponse:*

---
*Réservé pour corrections*
<div class="alert alert-block alert-info">
<b>Corrections: /5 </b>
</div>

Commentaires: 

FIN